In [ ]:
# Import libraries
import os 
import ee
import time
import rasterio
import math
import pyproj
import libpysal
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
from pylandtemp import single_window
from shapely.geometry import Point
from shapely.ops import unary_union

In [ ]:
directory = os.getcwd() #Location where files will be saved
c = "/"

In [ ]:
# Download street network
G = ox.graph_from_place('Greater Manchester', network_type='drive')

# Save to geopackage
ox.save_graph_geopackage(G, 'network.gpkg')

# Save to graphml
ox.io.save_graphml(G, 'network.graphml')

In [ ]:
# Calculating distance variabels to POIs

bus_dist_list = []

city = 'Greater Manchester'
espg = 'EPSG:27700'
city_path = directory + c + city

# Load in Manchester grid data
df = gpd.read_file('popgrid.gpkg')
df = df.to_crs(espg)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Load in street network graph
G = ox.load_graphml('network.graphml')

# Load in bus stops
busstop = gpd.read_file('bus_stops.gpkg').to_crs(espg)

# Import points as gdf and loop through them
for index, point in gdf.iterrows():
    polygon_index = busstop.distance(point.geometry).sort_values().index[0]
    nearest_bus = busstop.loc[polygon_index].geometry.centroid

    # Extract coordinates of point
    x, y = point.geometry.centroid.coords.xy
    x = x[0]
    y = y[0]

    # Find closest node
    node_counter = ox.distance.nearest_nodes(G, x, y)
        
    # Extract coordinates of point
    xx, yy = nearest_bus.xy
    xx = xx[0]
    yy = yy[0]
        
    # Find closest node
    node_bus = ox.distance.nearest_nodes(G, xx, yy)

    # Calculate shortest path
    try:
        nx.shortest_path_length(G, node_counter, node_bus)
        
    except:
        s_path = 0
        
    else:
        s_path = nx.shortest_path_length(G, node_counter, node_bus)

    bus_dist_list.append(s_path)

print(city + ' is done')

df['dist_to_busstop'] = bus_dist_list
df

In [ ]:
df.to_file('popgriddistance.gpkg', driver='GPKG')